In [ ]:
#Traducto Ingles a Español
#Alejandro Pardo/Michael lisker
#INSTALACIONES NECESARIAS

!pip install kagglehub --quiet

# IMPORTACIONES
import os
import re
import string
import random
import kagglehub
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization, MultiHeadAttention, Dense, LayerNormalization


In [ ]:
# DESCARGA DEL DATASET DESDE KAGGLEHUB
path = kagglehub.dataset_download("tejasurya/eng-spanish")
print("Archivos disponibles:", os.listdir(path))

# LECTURA DEL ARCHIVO .txt
archivo = os.path.join(path, "spa.txt")
with open(archivo, encoding="utf-8") as f:
    ejemplos = f.read().split("\n")

# PROCESAMIENTO DEL DATASET
dataset = []
for linea in ejemplos:
    partes = linea.split("\t")
    if len(partes) >= 2:
        ingles = partes[0]
        espanol = "[start] " + partes[1] + " [end]"
        dataset.append((ingles, espanol))

# DIVISIÓN DEL DATASET
random.shuffle(dataset)
val_split = int(0.15 * len(dataset))
train_pairs = dataset[:-2 * val_split]
val_pairs = dataset[-2 * val_split:-val_split]
test_pairs = dataset[-val_split:]

100%|██████████| 10.3M/10.3M [00:00<00:00, 12.4MB/s]

Extracting files...


Archivos disponibles: ['spa.txt', 'spa-eng']


Explicación:

Descarga del dataset: Usamos KaggleHub para descargar el dataset de traducción entre inglés y español.

Lectura del archivo: Se lee el archivo spa.txt que contiene las frases de inglés y sus respectivas traducciones al español.

Procesamiento de datos: El archivo es procesado línea por línea, y las frases en inglés y español se separan. Las traducciones al español se marcan con los tokens [start] al principio y [end] al final, lo cual es útil para marcar los límites de las traducciones en el modelo.

División del dataset: El dataset se divide en tres partes:

Entrenamiento (train): 70% de los datos.

Validación (val): 15% de los datos.

Prueba (test): 15% de los datos.

Se utiliza random.shuffle para aleatorizar el dataset antes de la división.

In [ ]:
# LIMPIEZA Y VECTORIZACIÓN
caracteres_a_eliminar = string.punctuation + "¿"
caracteres_a_eliminar = caracteres_a_eliminar.replace("[", "").replace("]", "")

def estandarizacion(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(lowercase, f"[{re.escape(caracteres_a_eliminar)}]", "")

vocab_size = 15000
sequence_length = 20

vectorizacion_entrada = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)

vectorizacion_salida = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=estandarizacion,
)

# DATOS DE TEXTO
train_entrada_texts = [pair[0] for pair in train_pairs]
train_salida_texts = [pair[1] for pair in train_pairs]
val_entrada_texts = [pair[0] for pair in val_pairs]
val_salida_texts = [pair[1] for pair in val_pairs]

# ADAPTACIÓN DE VECTORIZADORES
vectorizacion_entrada.adapt(train_entrada_texts)
vectorizacion_salida.adapt(train_salida_texts)


Explicación:

Limpieza de datos: La función estandarizacion elimina los caracteres no deseados (como puntuaciones y caracteres especiales) y convierte el texto a minúsculas. Esto facilita la tokenización y reduce la complejidad del texto.

Vectorización:

Se utiliza TextVectorization para convertir las frases en secuencias de enteros (tokens).

vocab_size define el tamaño del vocabulario, limitando a las 15,000 palabras más frecuentes.

sequence_length define la longitud máxima de las secuencias de entrada y salida. Si una secuencia es más larga, se truncará, y si es más corta, se rellenará.

vectorizacion_entrada se usa para transformar las frases en inglés, mientras que vectorizacion_salida se usa para las traducciones al español, agregando la funcionalidad de start y end tokens.

In [ ]:
# FUNCIONES PARA FORMATEAR LOS DATOS
def formato(inputs, targets):
    entrada = vectorizacion_entrada(inputs)
    salida = vectorizacion_salida(targets)
    return {"entrada": entrada, "salida": salida[:, :-1]}, salida[:, 1:]

batch_size = 128

train_ds = tf.data.Dataset.from_tensor_slices((train_entrada_texts, train_salida_texts))
train_ds = train_ds.batch(batch_size).map(formato).prefetch(tf.data.AUTOTUNE)

val_ds = tf.data.Dataset.from_tensor_slices((val_entrada_texts, val_salida_texts))
val_ds = val_ds.batch(batch_size).map(formato).prefetch(tf.data.AUTOTUNE)

# POSICIONAL EMBEDDING
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.position_embeddings = layers.Embedding(input_dim=sequence_length, output_dim=embed_dim)
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def get_config(self):
        config = super().get_config()
        config.update({
            "sequence_length": self.sequence_length,
            "vocab_size": self.vocab_size,
            "embed_dim": self.embed_dim,
        })
        return config


Explicación:

Positional Embedding: Esta capa agrega información de la posición de las palabras en la secuencia. Los Transformers no tienen una estructura secuencial explícita, por lo que necesitamos agregar información de la posición de cada palabra en la secuencia para que el modelo pueda entender el orden.

token_embeddings: Convierte los tokens (palabras) en vectores de alta dimensión.

position_embeddings: Asocia a cada posición de la secuencia un embedding.

Finalmente, se suma embedded_tokens y embedded_positions para obtener la representación final de la secuencia.

In [ ]:
# ENCODER
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.attention = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential([
            Dense(dense_dim, activation="relu"),
            Dense(embed_dim),
        ])
        self.layernorm_1 = LayerNormalization()
        self.layernorm_2 = LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)



Explicación:

Encoder Transformer: El encoder toma la secuencia de entrada (en inglés) y la convierte en una representación de alta dimensión.

MultiHeadAttention: Aplica atención de múltiples cabezas para capturar relaciones entre palabras en diferentes posiciones.

LayerNormalization: Normaliza las capas para estabilizar el entrenamiento y acelerar la convergencia.

Proyecto Denso: Después de la atención, se pasa por una capa densa para aumentar la capacidad de representación del modelo.

In [ ]:
# DECODER
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.attention_1 = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential([
            Dense(dense_dim, activation="relu"),
            Dense(embed_dim),
        ])
        self.layernorm_1 = LayerNormalization()
        self.layernorm_2 = LayerNormalization()
        self.layernorm_3 = LayerNormalization()
        self.supports_masking = True

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, seq_length = input_shape[0], input_shape[1]
        i = tf.range(seq_length)[:, tf.newaxis]
        j = tf.range(seq_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, seq_length, seq_length))
        mult = tf.concat([[batch_size], [1], [1]], axis=0)
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        else:
            padding_mask = mask

        attention_output_1 = self.attention_1(inputs, inputs, inputs, attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)

        attention_output_2 = self.attention_2(attention_output_1, encoder_outputs, encoder_outputs, attention_mask=padding_mask)
        attention_output_2 = self.layernorm_2(attention_output_1 + attention_output_2)

        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)


Explicación:

Decoder Transformer: El decoder genera la secuencia de salida (en español) a partir de la representación del encoder.

Utiliza atención similar al encoder, pero también aplica más de una capa de atención para procesar la secuencia generada.

Causal Attention Mask: Asegura que durante la generación, el modelo no vea futuras palabras en la secuencia.

In [ ]:
# CONSTRUCCIÓN DEL MODELO TRANSFORMER
embed_dim = 128
dense_dim = 512
num_heads = 4

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="entrada")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="salida")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)

transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)


Explicación:

Definición de la arquitectura completa: Aquí se está construyendo el modelo Transformer.

embed_dim, dense_dim, num_heads: Se definen los parámetros clave de la arquitectura, como la dimensión de los embeddings, el tamaño de las capas densas y el número de cabezas de la atención.

Encoder: La entrada (en inglés) se pasa a través de un PositionalEmbedding y luego se procesa por el TransformerEncoder.

Decoder: La entrada del decoder (en español) también pasa por un PositionalEmbedding y luego por el TransformerDecoder.

Dropout: Se utiliza Dropout para prevenir el sobreajuste durante el entrenamiento.

Dense: Finalmente, una capa densa se aplica para obtener la salida, que es una distribución de probabilidad sobre el vocabulario en español, utilizando softmax para obtener las probabilidades de cada palabra en la secuencia de salida.

In [ ]:
# COMPILACIÓN Y ENTRENAMIENTO
transformer.compile(optimizer="rmsprop",
                    loss="sparse_categorical_crossentropy",
                    metrics=["accuracy"])  # Solo se usa 'accuracy'

transformer.fit(train_ds,
                epochs=30,
                validation_data=val_ds,
                validation_freq=1)  # La precisión de validación se mostrará automáticamente



Epoch 1/30
761/761 ━━━━━━━━━━━━━━━━━━━━ 66s 63ms/step - accuracy: 0.6973 - loss: 2.8979 - val_accuracy: 0.7639 - val_loss: 1.4977
Epoch 2/30
761/761 ━━━━━━━━━━━━━━━━━━━━ 33s 43ms/step - accuracy: 0.7801 - loss: 1.4268 - val_accuracy: 0.8169 - val_loss: 1.1153
Epoch 3/30
761/761 ━━━━━━━━━━━━━━━━━━━━ 33s 43ms/step - accuracy: 0.8276 - loss: 1.0883 - val_accuracy: 0.8482 - val_loss: 0.9088
Epoch 4/30
761/761 ━━━━━━━━━━━━━━━━━━━━ 33s 43ms/step - accuracy: 0.8502 - loss: 0.9214 - val_accuracy: 0.8540 - val_loss: 0.8547
Epoch 5/30
761/761 ━━━━━━━━━━━━━━━━━━━━ 33s 43ms/step - accuracy: 0.8626 - loss: 0.8251 - val_accuracy: 0.8658 - val_loss: 0.7787
Epoch 6/30
761/761 ━━━━━━━━━━━━━━━━━━━━ 33s 43ms/step - accuracy: 0.8715 - loss: 0.7590 - val_accuracy: 0.8691 - val_loss: 0.7564
Epoch 7/30
761/761 ━━━━━━━━━━━━━━━━━━━━ 41s 43ms/step - accuracy: 0.8777 - loss: 0.7131 - val_accuracy: 0.8753 - val_loss: 0.7224
Epoch 8/30
761/761 ━━━━━━━━━━━━━━━━━━━━ 33s 43ms/step - accuracy: 0.8832 - loss: 0.6765 - 

Explicación:

Compilación del modelo:

El modelo se compila utilizando rmsprop como optimizador. RMSprop es comúnmente usado para redes neuronales profundas porque es adecuado para el entrenamiento con gradientes estocásticos.

Se utiliza sparse_categorical_crossentropy como la función de pérdida. Esto es adecuado para tareas de clasificación multiclase, como la traducción automática, donde cada palabra generada es tratada como una clase.

accuracy se usa como la métrica de evaluación, ya que estamos interesados en la precisión de la predicción de palabras.

Entrenamiento:

El modelo se entrena durante 30 épocas (ajustables según el rendimiento). Durante cada época, se calcula la pérdida y la precisión.

Se usa el dataset de entrenamiento (train_ds) y se valida el modelo con val_ds. La precisión de validación se muestra automáticamente al final de cada época.

In [ ]:
# CELDA PARA TRADUCIR UNA PALABRA INGRESADA

# Función para traducir una palabra en inglés al español
def traducir_palabra(entrada_texto):
    # Preprocesar la entrada (vectorizarla)
    entrada_vectorizada = vectorizacion_entrada([entrada_texto])

    # Iniciar la secuencia de salida con el token [start]
    decoder_input = tf.constant([[vectorizacion_salida.vocabulary_size() - 2]])  # [start] token

    # Generar la traducción palabra por palabra
    traduccion = []
    for _ in range(sequence_length):  # Limitar la longitud de la traducción
        # Hacer la predicción
        prediccion = transformer.predict([entrada_vectorizada, decoder_input])

        # Obtener la siguiente palabra (con el índice con mayor probabilidad)
        prediccion_idx = tf.argmax(prediccion[0], axis=-1)[-1].numpy()

        # Convertir el índice en palabra
        palabra_predicha = vectorizacion_salida.get_vocabulary()[prediccion_idx]

        # Si la predicción es el token [end], terminamos
        if palabra_predicha == "[end]":
            break

        # Añadir la palabra predicha a la traducción
        traduccion.append(palabra_predicha)

        # Actualizar la entrada del decoder con la palabra predicha
        decoder_input = tf.concat([decoder_input, tf.constant([[prediccion_idx]])], axis=-1)

    # Unir las palabras en una cadena
    return " ".join(traduccion)

# Interfaz de entrada de la palabra a traducir
entrada_usuario = input("Introduce una palabra o frase en inglés para traducir: ")

# Traducir y mostrar la traducción
traduccion = traducir_palabra(entrada_usuario)
print(f"Traducción al español: {traduccion}")


Introduce una palabra o frase en inglés para traducir: i have a great idea and i am a big genious
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step   
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Traducción al español: tengo una gran idea y yo soy muy muy bien


Explicación:

Función de traducción:

La función traducir_palabra toma como entrada una frase en inglés y genera la traducción correspondiente en español.

La entrada se vectoriza usando el vectorizador entrenado para las entradas (inglés).

Se inicia la traducción con el token [start], y luego el modelo genera la siguiente palabra en español, una por una, utilizando el decoder.

Si el modelo predice el token [end], la traducción se detiene.

La predicción se hace palabra por palabra, actualizando el decoder_input con cada nueva palabra generada.

Interfaz de usuario: Se le solicita al usuario que ingrese una frase en inglés, y la función devuelve la traducción al español.